In [1]:
import pandas as pd
import numpy as np
from Classes import Sortino

/Users/adrianlopez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/adrianlopez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
2025-06-06 20:58:29.140143: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
prices_df = pd.read_excel("daily_dbs/daily_dbs_sp50&nasdaq.xlsx")
market_features_df = pd.read_excel("daily_dbs/daily_market_features.xlsx")
rfr_df = pd.read_csv('daily_dbs/risk_free_rate.csv')

primero vamos a llenar los NaNs que tiene el archivo de risk free rate

In [3]:
rfr_df.columns = [col.strip() for col in rfr_df.columns]
rfr_df.rename(columns={"observation_date": "Date", "DGS10": "rfr"}, inplace=True)
rfr_df['Date'] = pd.to_datetime(rfr_df['Date'])

rfr_df = rfr_df.sort_values("Date")
rfr_df['rfr'] = rfr_df['rfr'].fillna(method='ffill')

rfr_df.to_csv('daily_dbs/risk_free_rate.csv', index=False)

/var/folders/g9/36zlnbj94qbbbsytd9nvq9c40000gn/T/ipykernel_29486/121179686.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rfr_df['rfr'] = rfr_df['rfr'].fillna(method='ffill')


Vamos a generar el dataset de portafolios con pesos aleatorios y vamos a calcular su rendimiento

In [4]:
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
prices_df.set_index('Date', inplace=True)
returns_df = np.log(prices_df/prices_df.shift(1)).dropna()
returns_df.to_csv('daily_dbs/returns_df.csv')

selección aleatoria de activos

In [2]:
returns_df = pd.read_csv("daily_dbs/returns_df.csv")

np.random.seed(42)
n = 20
tickers = returns_df.columns.tolist()
selected_assets = np.random.choice(tickers, n, replace=False).tolist()
selected_assets

['LIN',
 'KIM',
 'TT',
 'AME',
 'IT',
 'MO',
 'DTE',
 'DOV',
 'EXPE',
 'PG',
 'V',
 'COST',
 'KLAC',
 'ZBH',
 'KO',
 'GILD',
 'WSM',
 'ALL',
 'EQIX',
 'INTC']

In [6]:
# Clasificador entrenado (test de funcionalidad del classifier nuevo)

clf = PortfolioClassifier("market_caps.xlsx")

# Agregar portafolios de referencia
clf.add_reference_portfolio(clf.get_random_portfolio(n=20))
clf.add_reference_portfolio(clf.get_random_portfolio(n=20))

# Obtener portafolio temporal de prueba
x_temp = clf.get_random_portfolio(n=20)
print("Portafolio temporal:", x_temp)

# Clasificar
etiqueta = clf.classify(x_temp)
print

NameError: name 'PortfolioClassifier' is not defined

In [ ]:
# Traslado hacia los 3 svr's (high, mid o low cap)



llamamos a la clase de sortino que hace toda la chamba :V

In [3]:
rfr_path = 'daily_dbs/risk_free_rate.csv'
sortino = Sortino(returns_df, rfr_path, selected_assets)

sortino.generate_multiple_weights(num_combinations=10)
sortino.calculate_portfolio_returns()
sortino.calculate_sortino_ratio()

sortino_dataset = sortino.create_portfolio_dataset()
sortino_dataset.head()

Generadas 10 combinaciones de pesos para cada fecha.
Rendimientos del portafolio calculados para cada combinación de pesos.
Ratios de Sortino calculados para cada combinación de pesos.


,Date,Weight_LIN,Weight_KIM,Weight_TT,Weight_AME,Weight_IT,Weight_MO,Weight_DTE,Weight_DOV,Weight_EXPE,...,Weight_KLAC,Weight_ZBH,Weight_KO,Weight_GILD,Weight_WSM,Weight_ALL,Weight_EQIX,Weight_INTC,Portfolio_Returns,Sortino_Ratio
0,2010-01-05,7.724571,6.253236,7.952588,5.272664,8.835188,0.577477,6.850378,1.919994,1.454274,...,3.768831,11.854375,2.121907,0.207690,9.238500,9.765539,4.191094,5.623643,0.003115,0.327704
1,2010-01-05,5.212175,0.385506,7.613594,7.112527,2.092764,0.122766,7.487581,4.019103,4.326629,...,7.574336,6.956256,5.104926,6.424833,5.431921,4.599277,1.030769,6.507093,-0.000919,-0.118128
2,2010-01-05,8.388663,6.398421,8.386490,6.659546,2.112749,2.800460,2.193553,3.856382,0.398285,...,3.939565,6.967527,3.481900,2.664847,5.070547,7.082798,3.560734,9.574516,-0.000531,-0.075221
3,2010-01-05,0.393252,4.194272,9.710096,5.499139,4.249717,5.705351,5.779656,7.342407,1.281423,...,5.638579,2.394424,6.822541,7.425309,2.390804,3.790675,5.362211,4.983210,-0.003312,-0.382592
4,2010-01-05,3.925158,2.998488,1.007285,0.538831,9.656705,8.534437,3.575448,9.639170,6.818033,...,0.923862,6.069215,5.578211,2.143100,9.532317,7.871068,1.143085,9.378528,0.002868,0.300405


merge con los market features

In [4]:
sortino_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37730 entries, 0 to 37729
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               37730 non-null  datetime64[ns]
 1   Weight_LIN         37730 non-null  float64       
 2   Weight_KIM         37730 non-null  float64       
 3   Weight_TT          37730 non-null  float64       
 4   Weight_AME         37730 non-null  float64       
 5   Weight_IT          37730 non-null  float64       
 6   Weight_MO          37730 non-null  float64       
 7   Weight_DTE         37730 non-null  float64       
 8   Weight_DOV         37730 non-null  float64       
 9   Weight_EXPE        37730 non-null  float64       
 10  Weight_PG          37730 non-null  float64       
 11  Weight_V           37730 non-null  float64       
 12  Weight_COST        37730 non-null  float64       
 13  Weight_KLAC        37730 non-null  float64       
 14  Weight

In [6]:
database_daily = pd.merge(sortino_dataset, market_features_df, on="Date", how="inner")

In [7]:
database_daily.head()

,Date,Weight_LIN,Weight_KIM,Weight_TT,Weight_AME,Weight_IT,Weight_MO,Weight_DTE,Weight_DOV,Weight_EXPE,...,Sortino_Ratio,Treasury Bond 10Y,Treasury Bond 3M,WTI Index,Yield Curve Spread (10Y - 2Y),USD Index,MOM,VVIX,SMB,HML
0,2010-01-05,7.724571,6.253236,7.952588,5.272664,8.835188,0.577477,6.850378,1.919994,1.454274,...,0.327704,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
1,2010-01-05,5.212175,0.385506,7.613594,7.112527,2.092764,0.122766,7.487581,4.019103,4.326629,...,-0.118128,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
2,2010-01-05,8.388663,6.398421,8.386490,6.659546,2.112749,2.800460,2.193553,3.856382,0.398285,...,-0.075221,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
3,2010-01-05,0.393252,4.194272,9.710096,5.499139,4.249717,5.705351,5.779656,7.342407,1.281423,...,-0.382592,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
4,2010-01-05,3.925158,2.998488,1.007285,0.538831,9.656705,8.534437,3.575448,9.639170,6.818033,...,0.300405,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05


In [8]:
database_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37700 entries, 0 to 37699
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Date                           37700 non-null  datetime64[ns]
 1   Weight_LIN                     37700 non-null  float64       
 2   Weight_KIM                     37700 non-null  float64       
 3   Weight_TT                      37700 non-null  float64       
 4   Weight_AME                     37700 non-null  float64       
 5   Weight_IT                      37700 non-null  float64       
 6   Weight_MO                      37700 non-null  float64       
 7   Weight_DTE                     37700 non-null  float64       
 8   Weight_DOV                     37700 non-null  float64       
 9   Weight_EXPE                    37700 non-null  float64       
 10  Weight_PG                      37700 non-null  float64       
 11  Weight_V       